In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [3]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## Load Projects

In [4]:
projects = get_useful_projects()
for p in projects:
    print(p['story'])

creek
minotaur
seeknumbers
sudoku
yosenabe
hop
lights


## Calculate Semantic Similarity

In [5]:
file = "SimilarityDataframe.csv"
if os.path.exists(file):
    df = pd.read_csv(file)
    print("Loaded Similarity Dataframe")
    if len(projects) != df.shape[0]:
        print("Calculating Similarity Dataframe")
        df = get_similarity_dataframe(projects)
        df.to_csv("SimilarityDataframe.csv", index=False)
        print("Loaded Similarity Dataframe")
else:
    print("Calculating Similarity Dataframe")
    df = get_similarity_dataframe(projects)
    df.to_csv("SimilarityDataframe.csv", index=False)

Loaded Similarity Dataframe


In [6]:
df.loc[:, df.columns != 'embeddings']

,stories,problems,cos_creek,cos_minotaur,cos_seeknumbers,cos_sudoku,cos_yosenabe,cos_hop,cos_lights
0,creek,The goal of the game is to blacken some cells ...,1.000,0.805,0.916,0.828,0.855,0.872,0.892
1,minotaur,There is a player in a labyrinth. The goal is ...,0.805,1.000,0.813,0.752,0.795,0.837,0.807
2,seeknumbers,"Given a square grid, the goal is to construct ...",0.916,0.813,1.000,0.838,0.875,0.913,0.901
3,sudoku,The task of this project is to solve a Sudoku ...,0.828,0.752,0.838,1.000,0.902,0.843,0.855
4,yosenabe,The task of this project is to solve the Japan...,0.855,0.795,0.875,0.902,1.000,0.872,0.870
5,hop,The goal of the puzzle is to find a path betwe...,0.872,0.837,0.913,0.843,0.872,1.000,0.894
6,lights,The puzzle involves illuminating a two-dimensi...,0.892,0.807,0.901,0.855,0.870,0.894,1.000


## Connect to the Database

In [7]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [8]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO experiments (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

## Run experiments by Version

In [7]:
def run_main_pipeline(projects,df,conn,version=1):
    for project in projects:
        query = dict(project)
        example_1, example_2 = get_best_context(df,projects,project["story"])
        story = query["story"]
        story_1 = example_1["story"]
        story_2 = example_2["story"]
        print("Working Pipeline for story " + story + ". Context: " + story_1 + " and " + story_2)
        answer, prompts = pipeline(example_1,example_2,query,version=version)
        name_of_file = story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version) + "_" + date_time + ".lp"
        path_of_file = os.path.join("generated_solutions", name_of_file)
        save_in_database(conn,prompts[0].replace("'",""), prompts[1].replace("'",""), answer.replace("'",""), story, story_1, story_2, version, name_of_file)
        with open(path_of_file,"w") as file:
            file.write(answer)

In [10]:
version = 2
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [13]:
version = 3
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [10]:
version = 4
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [16]:
version = 9
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [12]:
version = 10
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


## All against all

In [11]:
def try_pipeline(example_1,example_2,query,version, iteration=0):
    story = query["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    if iteration == 4:
        raise Exception(f"It was not possible to have an encoding for story {story} from {story_1} and {story_2}. Too many tries to connect to the internet. Check your connection and try it again.") 
    else:
        iteration += 1
        try:
            answer, prompts = pipeline(example_1,example_2,query,version)
            return answer, prompts
        except:
            print(f"|-- Failed in Iteration {iteration}")
            time.sleep(40 + 10*iteration)
            answer, prompts = try_pipeline(example_1,example_2,query,version,iteration)
            return answer, prompts

In [13]:
versions = [3] ##schon: 2,3,4,9,10
for version in versions:
    for project in projects:
        if project in [creek, sudoku, minotaur, seeknumbers]: #, sudoku, yosenabe]:
            continue
        for example_1 in projects:
            if example_1 == project:
                continue
            if project == yosenabe and example_1 in [creek, minotaur, seeknumbers, sudoku]: #sudoku, yosenabe]:
                continue
            #if example_1 != sudoku:
            #    continue
            for example_2 in projects:
                if example_2 == project or example_2 == example_1:
                    continue
                if project == yosenabe and example_1 == hop and example_2 in [creek]:
                    continue
                #if example_1 == sudoku and example_2 in [creek]:#, seeknumbers, yosenabe]:
                #    continue
                query = dict(project)
                story = query["story"]
                story_1 = example_1["story"]
                story_2 = example_2["story"]
                ct = datetime.now()
                print(f"Version: {version}. Story {story}. Context: {story_1} and {story_2}. \t Timestamp: {ct}")
                answer, prompts = try_pipeline(example_1,example_2,query,version=version)
                name_of_file = story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version) + "_" + date_time + ".lp"
                path_of_file = os.path.join("generated_solutions", name_of_file)
                save_in_database(conn,prompts[0].replace("'",""), prompts[1].replace("'",""), answer.replace("'",""), story, story_1, story_2, version, name_of_file)
                with open(path_of_file,"w") as file:
                    file.write(answer)
                time.sleep(1)

Version: 3. Story yosenabe. Context: hop and minotaur. 	 Timestamp: 2023-12-04 18:33:58.508095
Version: 3. Story yosenabe. Context: hop and seeknumbers. 	 Timestamp: 2023-12-04 18:34:56.353044
Version: 3. Story yosenabe. Context: hop and sudoku. 	 Timestamp: 2023-12-04 18:36:21.085547
Version: 3. Story yosenabe. Context: hop and lights. 	 Timestamp: 2023-12-04 18:37:21.399797
Version: 3. Story yosenabe. Context: lights and creek. 	 Timestamp: 2023-12-04 18:38:14.380921
Version: 3. Story yosenabe. Context: lights and minotaur. 	 Timestamp: 2023-12-04 18:47:01.173759
Version: 3. Story yosenabe. Context: lights and seeknumbers. 	 Timestamp: 2023-12-04 18:47:49.355794
Version: 3. Story yosenabe. Context: lights and sudoku. 	 Timestamp: 2023-12-04 18:49:30.364612
Version: 3. Story yosenabe. Context: lights and hop. 	 Timestamp: 2023-12-04 18:50:32.561207
Version: 3. Story hop. Context: creek and minotaur. 	 Timestamp: 2023-12-04 18:51:48.341806
Version: 3. Story hop. Context: creek and seek